In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer


In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('F:/notebook_working/kaggle_compe/train.csv').fillna('No data')
test = pd.read_csv('F:/notebook_working/kaggle_compe/test.csv').fillna('No data')

#train.id = train.id.astype('str')
#train.comment_text = train.comment_text.astype('str')

#test.id = test.id.astype('str')
#test.comment_text = test.comment_text.astype('str')




In [3]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])


In [21]:
#train_text = train[['comment_text']]
#test_text = test[['comment_text']]
#all_text = pd.concat([train_text, test_text])

In [4]:
print train_text.shape
print test_text.shape
print train.info()
print test.info()
print all_text.shape
#print all_text

(159571,)
(153164,)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 8.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 2 columns):
id              153164 non-null object
comment_text    153164 non-null object
dtypes: object(2)
memory usage: 1.2+ MB
None
(312735,)


In [22]:
#train_text = map(lambda x:unicode(x, 'utf-8'), train_text)


In [24]:
#test_text = map(lambda x:unicode(x, 'utf-8', errors='ignore'), test_text)

In [5]:
#dont use errors='ignore'
#all_text.comment_text = map(lambda x:unicode(x, 'utf-8'), all_text.comment_text)
all_text = map(lambda x:unicode(x, 'utf-8'), all_text)

In [35]:
type(all_text)

list

In [6]:
#extracting the stopwords from nltk library
sw = stopwords.words('english')
# displaying the stopwords
np.array(sw)

array([u'i', u'me', u'my', u'myself', u'we', u'our', u'ours',
       u'ourselves', u'you', u"you're", u"you've", u"you'll", u"you'd",
       u'your', u'yours', u'yourself', u'yourselves', u'he', u'him',
       u'his', u'himself', u'she', u"she's", u'her', u'hers', u'herself',
       u'it', u"it's", u'its', u'itself', u'they', u'them', u'their',
       u'theirs', u'themselves', u'what', u'which', u'who', u'whom',
       u'this', u'that', u"that'll", u'these', u'those', u'am', u'is',
       u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has',
       u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an',
       u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until',
       u'while', u'of', u'at', u'by', u'for', u'with', u'about',
       u'against', u'between', u'into', u'through', u'during', u'before',
       u'after', u'above', u'below', u'to', u'from', u'up', u'down',
       u'in', u'out', u'on', u'off', u'over', u'under', u'again',
       u'further'

In [7]:
def stopwords(text):
    '''a function for removing the stopword'''
# removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

In [ ]:
#####all_text['comment_text'] = all_text['comment_text'].apply(stopwords)
#all_text.head(10)
all_text = map(stopwords, all_text)
all_text

In [9]:
PunctuationToRemove = [".", ",", ":", ";", "!" ,"?", "&", "\"", "\'", "~"]

In [ ]:
#s_removed_punctuations = s.translate(None, ''.join(PunctuationToRemove))
#all_text.comment_text = map(lambda x:x.translate(None, ''.join(PunctuationToRemove)), all_text.comment_text)
#all_text.head(10)

In [14]:
#dont run
#def remove_punctuation(text):
#    '''a function for removing punctuation'''
#    import string
# replacing the punctuations with no space,
# which in effect deletes the punctuation marks
 #   translator = string.maketrans('', '', string.punctuation)
# return the text stripped of punctuation marks
 #   return text.translate(translator)

In [10]:
def remove_punctuation(text):
    '''a function for removing punctuation'''
    for char in PunctuationToRemove:
        text = text.replace(char,"")
    return text

In [11]:
#all_text['comment_text'] = all_text['comment_text'].apply(remove_punctuation)
#all_text.head(10)
all_text = map(remove_punctuation, all_text)
#all_text

In [44]:
# create an object of stemming function
stemmer = SnowballStemmer("english")
def stemming(text):
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

In [45]:
#Dont run. not working in this case
#comment['comment_text'] = unicode(comment['comment_text'], 'utf-8')
#all_text['comment_text'] = all_text['comment_text'].apply(stemming)
#all_text.head(10)
all_text  = map(stemming, all_text)

,comment_text
0,explan edit made usernam hardcor metallica fan...
1,daww match background colour im seem stuck wit...
2,hey man im realli tri edit war guy constant re...
3,cant make real suggest improv - wonder section...
4,you sir hero chanc rememb page that on
5,congratul well use tool well · talk
6,cocksuck piss around work
7,vandal matt shirvington articl revert pleas ag...
8,sorri word nonsens offens you anyway im intend...
9,align subject contrari dulithgow


In [22]:
#dont run more than once
#all_text = all_text.comment_text

In [45]:

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    encoding='utf-8',
    lowercase=True,
    min_df=0.00001,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    #use_idf=1, smooth_idf=1,
    max_features=None)

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

msg = "There are {} tokens in Comment_text if we use word"
print(msg.format(len(word_vectorizer.get_feature_names())))


There are 65672 tokens in Comment_text if we use word


In [12]:

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    encoding='utf-8',
    lowercase=True,
    min_df=0.00001,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    #use_idf=1, smooth_idf=1,
    max_features=None)

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

msg = "There are {} tokens in Comment_text if we use word"
print(msg.format(len(word_vectorizer.get_feature_names())))


MemoryError: 

In [ ]:
#dont run
char_vectorizer = TfidfVectorizer(
    #norm=None,
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(1,2), 
    #non_negative=True,
    #min_df = 0.00009)
    max_features=10000)
    
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

msg = "There are {} tokens in Comment_text if we use char"
print(msg.format(len(char_vectorizer.get_feature_names())))


In [29]:
#word_vectorizer.get_feature_names()[:100]

In [32]:
#col = [i for i in word_vectorizer.get_feature_names()]
#temp = pd.DataFrame(train_word_features.todense(), columns=col)
#temp


In [46]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])
#X_train, X_valid, y_train, y_valid = train_test_split(train_features, train_target, test_size=0.3, random_state=42)


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression()

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_word_features, train_target)
    #submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('F:\submission.csv', index=False)

CV score for class toxic is 0.969449631157
CV score for class severe_toxic is 0.986069110385
CV score for class obscene is 0.985650217587
CV score for class threat is 0.980191041916
CV score for class insult is 0.976405748916
CV score for class identity_hate is 0.973997557636
Total CV score is 0.978627217933


In [ ]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)

    classifier = LogisticRegression()

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('submission.csv', index=False)

In [9]:
submission[class_name] = classifier.predict_proba(test_features)[:, 1]
submission.to_csv('submission.csv', index=False)